In [1]:
from pyne import data, nucname
from pyne.material import Material
import pandas as pd

In [2]:
pkl62 = 'not-scaled_nuc62_activities.pkl'
nuc62 = pd.read_pickle(pkl62).drop(columns=['total']).reset_index(drop=True)
nuc62 = nuc62.loc[(nuc62['Burnup'] > 0) & (nuc62['CoolingTime'] > 0)]
list62 = nuc62.columns.tolist()
del list62[0:8]
#print(len(list62))

# arbitrary minimum max activity to filter out ultra-low activity (and zero activity) nucs
min_maxact = 1e-7
new_nuclist = []
for nuc in list62:
    maxact = nuc62[nuc].describe()['max']
    if maxact > min_maxact:
        new_nuclist.append(nuc)
#print(len(new_nuclist))
#print(new_nuclist)

# labels removed
nuc49 = nuc62[new_nuclist]

In [3]:
#Choosing rando idx for this analysis
idx = 29748
no_gams = []
gamma_dflist = []
for nuc in new_nuclist:
    mat = Material()
    act_bq = nuc49.iloc[idx][nuc] * 3.7e10
    mat.from_activity({nuc : act_bq})
    try:
        gamlist = mat.gammas()
        gamma = pd.DataFrame(gamlist, columns = ['energy', 'intensity'])
        num_atoms = mat.mass * 6.02e23 / data.atomic_mass(nucname.id(nuc))
        gamma['total_intensity'] = num_atoms * gamma['intensity']
        gamma['nucID'] = nuc
        gamma['half life (y)'] = data.half_life(nucname.id(nuc)) / 60 / 60 / 24 / 365.25
        gamma_dflist.append(gamma)
    except:
        no_gams.append(nuc)
all_gammas = pd.concat(gamma_dflist).reset_index(drop=True)

In [4]:
print(no_gams)

['ac227', 'cf252', 'th229', 'u233']


In [8]:
for nuc in nuc49:
    children = data.gamma_from_to_byparent(nucname.id(nuc))
    neg_from = [(x, y) for x, y in children if x < 0]
    # it was determined that negative "to" children don't impact mat.gammas()
    #neg_to = [y for x, y in children if y < 0]
    if (len(neg_from) > 0):
        print(nuc)
        print(nucname.id(nuc))
        print(neg_from)
        print('\n')

ac227
892270000
[(-872230000, 872230000)]


cf252
982520000
[(-581490000, 581490002), (-451100000, -451100000), (-461170000, 461170003), (-451100000, -451100000), (-431080000, 431080001), (-411020000, -411020000), (-431080000, 431080001), (-581490000, -581490000), (-380940000, 380940006), (-411020000, -411020000), (-581490000, -581490000), (-581490000, -581490000), (-581490000, 581490002), (-441130000, 441130009), (-441130000, -441130000), (-451100000, -451100000), (-411020000, -411020000), (-451100000, -451100000), (-411020000, -411020000), (-411020000, -411020000), (-411020000, -411020000), (-411020000, -411020000), (-411020000, -411020000), (-581490000, -581490000), (-411020000, -411020000), (-581490000, -581490000), (-451100000, -451100000), (-441130000, 441130008), (-441130000, -441130000), (-411020000, -411020000), (-411020000, 411020006), (-451100000, -451100000), (-411020000, -411020000), (-411020000, -411020000), (-451100000, -451100000), (-411020000, 411020001), (-581490000, 

In [9]:
nuclide = 'ac227'
parent = nucname.id(nuclide)
decay_c = data.decay_const(parent)
energies = data.gamma_energy(parent)
intensities = data.gamma_photon_intensity(parent)
children = data.gamma_from_to_byparent(parent)
decay_children = data.decay_data_children(parent)
decay_branches = data.decay_photon_branch_ratio_byparent(parent)

In [15]:
children[40:60]

[(872230012, 872230003),
 (872230017, 872230012),
 (872230015, 872230005),
 (872230017, 872230011),
 (872230007, 872230001),
 (-872230000, 872230000),
 (872230008, 872230001),
 (872230007, 872230000),
 (872230014, 872230003),
 (872230015, 872230003),
 (872230008, 872230000),
 (872230009, 872230001),
 (872230017, 872230010),
 (872230011, 872230001),
 (872230016, 872230003),
 (0, 0),
 (872230013, 872230001),
 (872230014, 872230001),
 (872230015, 872230001),
 (872230013, 872230000)]

In [16]:
decay_children

[872230000, 902270000]

In [17]:
# from pyne source code in data.cpp
gammas_result = []
for i, c in enumerate(children):
    for j, dc in enumerate(decay_children):
        # here is where we decide to skip negative nuclides
        if c[0] > 0:
            if nucname.zzzaaa(c[0]) == nucname.zzzaaa(dc):
                gammas_result.append((energies[i][0], decay_c*intensities[i][0]*decay_branches[j][0]))
gammas_result

[(9.3, 1.1097228374671527e-13),
 (12.9, 1.5314222049194625e-14),
 (15.2, 6.35568534185733e-13),
 (24.5, 2.82474904082548e-11),
 (25.95, 6.125688819677849e-16),
 (28.6, 4.2371235612382196e-11),
 (33.5, 1.22513776393557e-13),
 (35.0, 3.062844409838925e-14),
 (37.47, 3.062844409838925e-15),
 (37.9, 4.94331082144459e-11),
 (44.7, 1.22513776393557e-13),
 (51.06, 3.0628444098389247e-16),
 (52.32, 1.5314222049194626e-15),
 (53.7, 4.594266614758387e-14),
 (55.0, 4.90055105574228e-13),
 (55.8, 4.287982173774495e-15),
 (57.56, 3.5222710713147635e-15),
 (59.4, 4.594266614758387e-14),
 (60.6, 4.594266614758387e-14),
 (69.28, 4.2879821737744945e-12),
 (70.6, 9.188533229516774e-14),
 (72.5, 7.657111024597313e-14),
 (79.54, 1.22513776393557e-12),
 (82.2, 9.188533229516774e-13),
 (83.0, 1.5314222049194626e-15),
 (85.0, 1.22513776393557e-14),
 (86.1, 5.206835496726172e-13),
 (86.7, 3.062844409838925e-12),
 (88.1, 7.657111024597312e-13),
 (88.5, 1.0719955434436237e-15),
 (90.0, 1.990848866395301e-13),
 

In [18]:
nuclide = 'cs137'
parent = nucname.id(nuclide)
decay_c = data.decay_const(parent)
energies = data.gamma_energy(parent)
intensities = data.gamma_photon_intensity(parent)
children = data.gamma_from_to_byparent(parent)
decay_children = data.decay_data_children(parent)
decay_branches = data.decay_photon_branch_ratio_byparent(parent)

In [19]:
decay_branches

[(1.0, nan)]

In [20]:
children

[(561370001, 561370000), (561370002, 561370000)]

In [21]:
decay_children

[561370000]

In [22]:
gammas_result = []
for i, c in enumerate(children):
    for j, dc in enumerate(decay_children):
        # here is where we decide to skip negative nuclides
        if c[0] > 0:
            if nucname.zzzaaa(c[0]) == nucname.zzzaaa(dc):
                gammas_result.append((energies[i][0], decay_c*intensities[i][0]*decay_branches[j][0]))
gammas_result

[(283.5, 4.2351778792770855e-13), (661.657, 6.214028233215172e-08)]

# Result: Manual implementation of pyne gammas method 

In [23]:
def manual_gammas(nuclide):
    parent = nucname.id(nuclide)
    decay_c = data.decay_const(parent)
    energies = data.gamma_energy(parent)
    intensities = data.gamma_photon_intensity(parent)
    children = data.gamma_from_to_byparent(parent)
    decay_children = data.decay_data_children(parent)
    decay_branches = data.decay_photon_branch_ratio_byparent(parent)
    
    gammas_result = []
    for i, c in enumerate(children):
        for j, dc in enumerate(decay_children):
            # here is where we skip negative nuclides
            if c[0] > 0:
                if nucname.zzzaaa(c[0]) == nucname.zzzaaa(dc):
                    gammas_result.append((energies[i][0], decay_c*intensities[i][0]*decay_branches[j][0]))
    return gammas_result

In [24]:
idx = 29748
no_gams = []
gamma_dflist = []
for nuc in new_nuclist:
    mat = Material()
    act_bq = nuc49.iloc[idx][nuc] * 3.7e10
    mat.from_activity({nuc : act_bq})
    try:
        gamlist = manual_gammas(nuc)
        gamma = pd.DataFrame(gamlist, columns = ['energy', 'intensity'])
        num_atoms = mat.mass * 6.02e23 / data.atomic_mass(nucname.id(nuc))
        gamma['total_intensity'] = num_atoms * gamma['intensity']
        gamma['nucID'] = nuc
        gamma['half life (y)'] = data.half_life(nucname.id(nuc)) / 60 / 60 / 24 / 365.25
        gamma_dflist.append(gamma)
    except:
        no_gams.append(nuc)
all_gammas = pd.concat(gamma_dflist).reset_index(drop=True)

In [25]:
no_gams

[]

In [26]:
all_gammas['total_intensity'].describe()

count    1.708000e+03
mean     1.135661e+13
std      2.929383e+14
min      4.086126e-17
25%      2.581830e+00
50%      2.495861e+04
75%      2.501519e+07
max      1.040275e+16
Name: total_intensity, dtype: float64

In [27]:
all_gammas.loc[all_gammas['nucID'] == 'ac227']

,energy,intensity,total_intensity,nucID,half life (y)
0,9.30,1.109723e-13,0.406854,ac227,21.772
1,12.90,1.531422e-14,0.056146,ac227,21.772
2,15.20,6.355685e-13,2.330164,ac227,21.772
3,24.50,2.824749e-11,103.562842,ac227,21.772
4,25.95,6.125689e-16,0.002246,ac227,21.772
...,...,...,...,...,...
62,439.60,3.828556e-14,0.140365,ac227,21.772
63,441.00,6.125689e-14,0.224584,ac227,21.772
64,460.20,2.297133e-13,0.842190,ac227,21.772
65,527.60,3.215987e-14,0.117907,ac227,21.772
